In [ ]:
# !pip install git+https://github.com/cocodataset/cocoapi/#subdirectory=PythonAPI
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu110/torch1.7/index.html # gpu
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cpu/torch1.7/index.html # cpu
# !pip install opencv-python

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import cv2
import os
import json
import itertools
import random
import matplotlib.pyplot as plt

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.structures import BoxMode
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.modeling import build_model

In [ ]:
def create_dicts(ds_dir):
    f_for_json = os.path.join(ds_dir, "coco/data.json")
    with open(f_for_json) as f:
        img_anns = json.load(f)
    dataset_dicts = []

    for i,fn in enumerate(img_anns["images"]):
        obj={}
        obj["file_name"] = os.path.join(ds_dir,fn["file_name"])
        obj["height"] = fn["height"]
        obj["width"] = fn["width"]
        meta ={
            "bbox": img_anns["annotations"][i]["bbox"],
            "bbox_mode": BoxMode.XYWH_ABS,
            "segmentation": [],
            "category_id": img_anns["annotations"][i]["category_id"],
            "iscrowd": 0
        }
        obj["annotations"] = [meta]
        dataset_dicts.append(obj)
    return dataset_dicts

In [ ]:
SIZE = 128
ds = "ds/"

In [ ]:
for d in ["train", "val"]:
    DatasetCatalog.register(ds + d, lambda d=d: create_dicts(ds + d))
    MetadataCatalog.get(ds + d).set(thing_classes=["fist","mid","ok hand"])
ds_metadata = MetadataCatalog.get(ds+"train")

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (ds+"train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 500
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = SIZE
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train();

### Valid

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR,"model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
cfg.DATASETS.TEST = (ds+"val",)
predictor = DefaultPredictor(cfg)

In [ ]:
dataset_dicts = create_dicts(ds+"val")
for d in random.sample(dataset_dicts,3):
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:,:,::-1],metadata=ds_metadata,scale=.8)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize=(14,10))
    plt.imshow(cv2.cvtColor(v.get_image()[:,:,::-1],cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir output

In [ ]:
# print(cfg.dump())
f = open("config.yml","w")
f.write(cfg.dump())
f.close()

In [ ]:
cfg = get_cfg()
cfg.merge_from_file("config.yml")
predictor = DefaultPredictor(cfg)

In [ ]:
dataset_dicts = create_dicts("ds/val")
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], metadata=ds_metadata)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize = (14, 10))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
cfg = get_cfg()
cfg.merge_from_file("config.yml")
cfg.SOLVER.MAX_ITER = 1000

os.makedirs(cfg.OUTPUT_DIR,exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

In [ ]:
samples = random.sample(create_dicts("./ds/tmp/"), 16)
for d in samples:
    im = cv2.imread(d)
    outputs = predictor(im)
    v = Visualizer(im[:,:,::-1],metadata=ds_metadata)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize=(14,10))
    plt.imshow(cv2.cvtColor(v.get_image()[:,:,::-1],cv2.COLOR_BGR2RGB))
    plt.show()